# Execute Multiple Tests on Water Level Data

This Notebook loads a water level dataset and then uses the Configuration and Stream concepts from QARTOD v2 to run a series of tests.

The steps:

* Create a Bokeh-based function that can plot the results from any test
* Create a configuration string and convert that into a Config object
* Read data from a CSV file into a Pandas dataframe
* Make a Stream from that dataframe
* Run tests on the Stream using the Config and put the results in a list
* Plot all the results with the original data


In [ ]:
from __future__ import annotations

import pandas as pd
from bokeh.plotting import output_notebook

from joa_qartod_config.graphics import plot_results
from joa_qartod_config.qartod import choose_stream, config_generator, run_qartod

output_notebook()

In [ ]:
stream = choose_stream()

In [ ]:

output_config = config_generator(stream=stream.value)

# Specify data and QC configuration

In [ ]:
# QC configuration
reference_config_dict = {
    "contexts": [
        {
            "streams": {
                "sea_surface_height_above_sea_level": {
                    "qartod": {
                        "gross_range_test": {
                            "fail_span": [-10, 10],
                            "suspect_span": [-2, 3],
                        },
                        "flat_line_test": {
                            "tolerance": 0.001,
                            "suspect_threshold": 10800,
                            "fail_threshold": 21600
                        },
                        "rate_of_change_test": {
                            "threshold": 0.001
                        },
                        "spike_test": {
                            "suspect_threshold": 0.8,
                            "fail_threshold": 3
                        }
                    }
                }
            }
        }
    ]
}

# Create a high level and flexible configuration object
#context_configs = Config(output_config.value)

# Load data, run tests and plot results

In [ ]:

url = "https://github.com/ioos/ioos_qc/raw/master/docs/source/examples"
fname = f"{url}/water_level_example.csv"
variable_name = "sea_surface_height_above_sea_level"

data = pd.read_csv(fname, parse_dates=["time"])
data.head()

In [ ]:
title = "Water Level [MHHW] [m] : Kotzebue, AK"
results_list = run_qartod(data, output_config.value)

for test in ["gross_range_test", "rate_of_change_test"]:
  plot_results(data, variable_name, results_list, title, test)